In [1]:
import dash
from dash import dcc
from dash import html
from dash import dash_table
from dash import Dash, _dash_renderer
from dash import callback
from dash.dependencies import Input, Output, ClientsideFunction

import numpy as np
import pandas as pd
import pathlib
import plotly.express as px
import dash_mantine_components as dmc
_dash_renderer._set_react_version("18.2.0")

In [2]:
full_champion_data = pd.read_csv('csv/full_champion_data.csv', index_col=0)
champion_role_winrate = pd.read_csv('csv/champion_role_winrate.csv', index_col=0)
champion_role_pickrate = pd.read_csv('csv/champion_role_pickrate.csv', index_col=0)
game_duration = pd.read_csv('csv/game_duration.csv', index_col=0)
game_time_graph = pd.read_csv('csv/game_time_graph.csv', index_col=0)
winrate_objective = pd.read_csv('csv/winrate_objective.csv', index_col=0)

In [3]:
winrate_objective_color = [
    "indigo.6", "blue.6", "teal.6", "green.6", 
    "yellow.6", "orange.6", "red.6", "pink.6", "purple.6"
]

In [7]:
app = dash.Dash(
    __name__,
    meta_tags=[{"name": "viewport", "content": "width=device-width, initial-scale=1"}],
    external_stylesheets=dmc.styles.ALL
)
app.title = "LOL Analysis Project Data Visualization"

server = app.server
app.config.suppress_callback_exceptions = True

app.layout = dmc.MantineProvider(
    forceColorScheme="light",
    children=[
        dmc.Title("League of Legends Data Visualization and Analysis", id='dashboard-title', size="h3", style={"textAlign": "center"}),
        dmc.Grid([
            dmc.GridCol([
                dmc.RadioGroup(children=dmc.Group([dmc.Radio(i, value=i) for i in winrate_objective.columns]), 
                id='winrate-objective-radio', 
                value=winrate_objective.columns[0], 
                size='sm'),
                dash_table.DataTable(id='winrate-objective-table', data=winrate_objective.reset_index().to_dict('records'), style_table={'overflowX': 'auto'})
            ], span=6),
            dmc.GridCol([
                dmc.LineChart(
                    id='winrate-objective-linechart',
                    h=300,
                    data=winrate_objective.reset_index().to_dict('records'),
                    dataKey=winrate_objective.index,
                    series=[{"name": objective, "color": winrate_objective_color[i % len(winrate_objective_color)],}
                             for i, objective in enumerate(winrate_objective.columns)],
                    curveType="linear",
                    tickLine="xy",
                    xAxisLabel="Rank",
                    yAxisLabel="Winrate (%)",
                )
            ], span=6)
        ])
    ]
)

if __name__ == "__main__":
    app.run(debug=True)